In [1]:
import os

In [2]:
os.getcwd()

'C:\\Users\\student\\Documents\\Daikoku_Python'

In [3]:
import pandas as pd

In [4]:
data = pd.read_excel('call\\금통위data\\금통위 의사록2.xlsx', usecols = [1,2])

In [5]:
data.head()

,date,text
0,2005-06-09,〈의안 제21호 ― 통화정책방향〉(１) 전일 개최된 동향보고회의에서 조사국장이 보고...
1,2005-06-23,〈의안 제23호 ― 2005년 3/4분기 한국은행 총액한도대출의 한도 결정〉(１) ...
2,2005-07-07,〈의안 제25호 ― 통화정책방향〉(１) 전일 개최된 동향보고회의에서 조사국장이 보고...
3,2005-08-11,〈의안 제31호 ― 통화정책방향〉(１) 전일 개최된 동향보고회의에서 조사국장이 보고...
4,2005-09-08,〈의안 제32호 ― 통화정책방향〉(１) 전일 개최된 동향보고회의에서 조사국장이 보고...


In [6]:
import re
for i in range(len(data)):
    data.loc[i, 'text'] = re.sub('- \d+ -', '', data.loc[i, 'text'])
    data.loc[i, 'text'] = re.sub('―|｢|｣|-|[․/→←+]', '', data.loc[i, 'text'])

In [ ]:
a = data.loc[1, 'text']
a

In [9]:
def tidy_sentences(section):
    # 원래 코드 : r'((?<=[함음됨임봄짐움])(\s*\n|\.|;)|(?<=다)\.)\s*'
#     r'((?<=[함음됨임봄짐움])+[\s\n\.;\(]+|(?<=다)\.)\s+'
    sentence_enders = re.compile(r'((?<=[함음됨임봄짐움])+[ \s\n\.;\(]+|(?<=다)\.)\s*')
    splits = list((m.start(), m.end()) for m in re.finditer(sentence_enders, section))
    starts = [0] + [i[1] for i in splits]
    ends = [i[0] for i in splits]
    sentences = [section[start:end] for start, end in zip(starts[:-1], ends)]
    for i, s in enumerate(sentences):
        sentences[i] = (s.replace('\n', ' ').replace(' ', ' ')) + '.'

    text = '\n'.join(sentences) if len(sentences) > 0 else ''
    return sentences, text


import re 

def preprocess_minutes(minutes):

    pos = re.search('(.?국내외\s?경제\s?동향.?과 관련하여,?|\(가\).+경제전망.*|\(가\) 국내외 경제동향 및 평가)\n?\s*일부 위원은', minutes, re.MULTILINE)
    s1 = pos.start() if pos else -1
    pos = re.search('(.?외환.?국제금융\s?동향.?과 관련하여.*|\(나\) 외환.국제금융\s?(및 금융시장)?\s?동향)\n?\s*(일부 위원은|대부분의 위원들은)|｢?.?외환.?국제금융\s?동향.?｣?과 관련하여.*', minutes,re.MULTILINE)
    s2 = pos.start() if pos else -1
    pos = re.search('(.?금융시장\s?동향.?과 관련하여.?|\(다\) 금융시장\s?동향)\n?\s*일부 위원은|｢?.?금융시장\s?동향.?｣?과 관련하여,?', minutes, re.MULTILINE)
    s3 = pos.start() if pos else -1
    pos = re.search('((\((다|라)\) )?.?통화정책\s?방향.?에 관한 토론,?|이상과 같은 의견\s?교환을 바탕으로.*통화정책\s?방향.*에.*토론.*)\n?', minutes,re.MULTILINE)
    s4 = pos.start() if pos else -1
    pos = re.search('(\(4\) 정부측 열석자 발언.*)\n?', minutes, re.MULTILINE)
    s5 = pos.start() if pos else -1
    pos = re.search('(\(.*\) 한국은행 기준금리 결정에 관한 위원별 의견\s?개진|이상과 같은 토론에 이어 .* 관한 위원별 의견개진이 있었음.*)\n?', minutes,re.MULTILINE)
    s6 = pos.start() if pos else -1
    positer = re.finditer('(\(\s?.*\s?\) ()(심의결과|토의결론))\n?', minutes, re.MULTILINE)
    s7 = [pos.start() for pos in positer if pos.start() > s6]
    s7 = s7[0] if s7 else -1

    # 국내외 경제동향
    bos = s1
    eos = s2
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section1, section1_txt = tidy_sentences(section)

    # 외환․국제금융 동향
    bos = s2
    eos = s3 if s3 >= 0 else s4
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section2, section2_txt = tidy_sentences(section)
    #print(section)

    # 금융시장 동향
    bos = s3
    eos = s4
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section3, section3_txt = tidy_sentences(section)

    # 통화정책방향
    bos = s4
    eos = s5 if s5 >= 0 else s6 if s6 >= 0 else s7
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section4, section4_txt = tidy_sentences(section)

    # 위원별 의견 개진
    bos = s6
    eos = s7
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section5, section5_txt = tidy_sentences(section)

    # 정부측 열석자 발언
    bos = s5
    eos = s6
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('정부측 열석자 발언', section, re.MULTILINE)
    bos = pos.end() + 1 if pos else -1
    section = section[bos:] if bos >= 0 else section
    section6, section6_txt = tidy_sentences(section)

    sections = ['Economic Situation', 'Foreign Currency', 'Financial Markets',
                'Monetary Policy', 'Participants’ Views', 'Government’s View']
    section_texts = (section1, section2, section3, section4, section5, section6)


    return sections, section_texts

In [7]:
data.head()

,date,text
0,2005-06-09,〈의안 제21호 통화정책방향〉(１) 전일 개최된 동향보고회의에서 조사국장이 보고 ...
1,2005-06-23,〈의안 제23호 2005년 34분기 한국은행 총액한도대출의 한도 결정〉(１) 담당...
2,2005-07-07,〈의안 제25호 통화정책방향〉(１) 전일 개최된 동향보고회의에서 조사국장이 보고 ...
3,2005-08-11,〈의안 제31호 통화정책방향〉(１) 전일 개최된 동향보고회의에서 조사국장이 보고 ...
4,2005-09-08,〈의안 제32호 통화정책방향〉(１) 전일 개최된 동향보고회의에서 조사국장이 보고 ...


In [9]:
a = data.loc[1, 'text']
a

'〈의안 제23호  2005년 34분기 한국은행 총액한도대출의 한도 결정〉(１) 담당 부총재보가 중소기업대출 취급 원활화를 위하여 2005년 34분기 총액한도대출의 한도를 2005년 24분기 한도와 동일한 수준으로 유지하고자 한다는 내용의 제안설명을 하였음 (２) 위원 토의내용 일부 위원은 3월말 8.3조원이었던 총액한도대출 잔액이 4∼5월중 다소 늘어 5월말 현재는 8.6조원을 기록하였으나 이는 배정한도 9.4조억원 대비 약 8천억원 미달한 수준으로 지난 몇 달 동안 이 같은 한도와 실제 대출액간 차이를 줄이는 등 중소기업 지원을 위한 총액한도대출제도 개선에 관한 논의 제기가 여러 차례 있었는바 동 제도의 개선방안을 적극 검토할 필요가 있다는 견해를 나타내었음이에 대해 관련부서에서는 금융기관의 중소기업 시설자금 대출 촉진을 위해 총액한도대출제도를 개선하는 방안을 전향적으로 검토하고 있으며, 아울러 실무적으로 한도 배정기준을 개선하여 총액한도대출 대상 은행에 대한 검사를 통해 적발된 대출 위규수혜 은행과 중소기업대출 의무비율 미달 은행에 대한 제재조치로 차감한 한도를 중소기업대출을 적극적으로 취급한 여타 은행 등에 추가 배정함으로써 한도와 실제 지원액 간의 차이를 줄이는 방안을 추진중에 있다고 설명하였음 동 위원은 중소기업 시설자금에 대한 총액한도대출 지원방안의 주요 내용에 관해 물었으며, 이에 대해 관련부서에서는 시설자금의 경우 만기가 최소 3∼5년 정도이므로 분기별로 금통위에서 정하는 한도를 금융기관의 중소기업에 대한 시설자금 대출과 직접 연계시키기엔 무리가 있어 일정 기간중 시설자금 대출 취급실적을 평가해 실적이 큰 은행에 대해서는 총액한도를 더 배정하는 것을 기본방향으로 검토하고 있다고 답변하였음 동 위원은 최근 한국은행이 외국환은행과의 외화대출 연계 통화스왑을 통해 몇 가지 용도의 기업대출을 지원하기로 했는데 이 부분도 간접적으로 한국은행의 대출행위에 해당한다고 볼 수 있으므로 정기적으로 동 거래에 대한 한도를 정하는 데 대한 검토가 필요할 것 같다는

In [8]:
def tidy_sentences(section):
    sentence_enders = re.compile(r'((?<=[함음됨임봄짐움])+[ \s\n\.;\(]+|(?<=다)\.)\s+')
    splits = list((m.start(), m.end()) for m in re.finditer(sentence_enders, section))
    starts = [0] + [i[1] for i in splits]
    ends = [i[0] for i in splits]
    sentences = [section[start:end] for start, end in zip(starts[:-1], ends)]
    for i, s in enumerate(sentences):
        sentences[i] = (s.replace('\n', ' ').replace(' ', ' ')) + '.'

    text = '\n'.join(sentences) if len(sentences) > 0 else ''
    return sentences, text

In [12]:
def txt_split(data):
    result_list = []
    for i in range(len(data)):
        time = data.loc[i, 'date']
        text = data.loc[i, 'text']

        p1 = re.search('(.?외환.?국제금융\s?동향.?과 관련하여.*|\(나\) 외환.국제금융\s?(및 금융시장)?\s?동향)\n?\s*(일부 위원은|대부분의 위원들은)|｢?.?외환.?국제금융\s?동향.?｣?과 관련하여.*' , text)
        p2 = re.search('(.?금융시장\s?동향.?과 관련하여,?|\(다\) 금융시장\s?동향)\n?\s*일부 위원은|｢?.?금융시장\s?동향.?｣?과 관련하여,?', text)
        p3 = re.search('((\((다|라)\) )?.?통화정책\s?방향.?에 관한 토론,?|이상과 같은 의견\s?교환을 바탕으로.*통화정책\s?방향.*에.*토론.*)\n?', text)
        
        s1 = p1.start() if p1 else -1
        s2 = p2.start() if p2 else -1
        s3 = p3.start() if p2 else -1
        
        section_2 = text[s1:s2] if s1 >= 0 or s2 >= 0 else ''
        section_3 = text[s2:s3] if s2 >= 0 or s3 >= 0 else ''
        
        sentences2, text2 = tidy_sentences(section_2)
        sentences3, text3 = tidy_sentences(section_3)
        new_dict = {
            'date' : time,
            's2' : text2,
            's3' : text3
        }
        result_list.append(new_dict)
        
    df = pd.DataFrame(result_list)
    return df

In [13]:
df_t = txt_split(data)

In [14]:
df_t.head()

,date,s2,s3
0,2005-06-09,"외환국제금융 동향과 관련하여, 일부 위원은 우리나라 경제가 일본경제에 비해 더 나...","금융시장 동향과 관련하여, 일부 위원은 M1이 실물경제와 어느 정도의 시차를 두고..."
1,2005-06-23,,
2,2005-07-07,"외환국제금융 동향과 관련하여, 일부 위원은 6월 들어 국제유가가 크게 오르고 원화...","금융시장 동향과 관련하여, 일부 위원은 단기적으로 대출이 급격히 증가한 후에는 반..."
3,2005-08-11,"외환국제금융 동향과 관련하여, 일부 위원은 위안화 절상 이후 아시아 통화중 우리나...","금융시장 동향과 관련하여, 일부 위원은 최근 장기시장금리 상승에도 불구하고 기업 ..."
4,2005-09-08,"외환국제금융 동향과 관련하여, 일부 위원은 최근 인도네시아 금융불안 사태가 발생한...","금융시장 동향과 관련하여, 일부 위원은 시장이 제대로 작동한다면 비우량등급의 회사..."


In [16]:
df_t.to_excel('금통위 문자열.xlsx')

# ngram 추가

In [30]:
df_t['call'] = 0
df_t['ngram_s2'] = 0
df_t['ngram_s3'] = 0
df_t.head()


,date,s2,s3,call,ngram_s2,ngram_s3
0,2005-06-09,"[ 외환국제금융 동향과 관련하여, 일부 위원은 우리나라 경제가 일본경제에 비해 더 ...","[ 금융시장 동향과 관련하여, 일부 위원은 M1이 실물경제와 어느 정도의 시차를 두...",0,0,0
1,2005-06-23,[],[],0,0,0
2,2005-07-07,"[ 외환국제금융 동향과 관련하여, 일부 위원은 6월 들어 국제유가가 크게 오르고 원...","[ 금융시장 동향과 관련하여, 일부 위원은 단기적으로 대출이 급격히 증가한 후에는 ...",0,0,0
3,2005-08-11,"[ 외환국제금융 동향과 관련하여, 일부 위원은 위안화 절상 이후 아시아 통화중 우리...","[ 금융시장 동향과 관련하여, 일부 위원은 최근 장기시장금리 상승에도 불구하고 기업...",0,0,0
4,2005-09-08,"[ 외환국제금융 동향과 관련하여, 일부 위원은 최근 인도네시아 금융불안 사태가 발생...","[ 금융시장 동향과 관련하여, 일부 위원은 시장이 제대로 작동한다면 비우량등급의 회...",0,0,0


In [16]:
df_t.loc[0, 's2']

[' 외환국제금융 동향과 관련하여, 일부 위원은 우리나라 경제가 일본경제에 비해 더 나아질 특별한 요인이 없음에도 불구하고 최근 원엔 환율이 지속적으로 하락하는 이유를 물었으며, 이에 대해 관련부서에서는 위안화 절상시 동아시아권 통화의 절상폭이 클 것이라는 기대로 원화나 대만달러가 엔달러 환율의 상승을 추수하지 못하고 국내 외환수급 면에서 공급우위 상태가 지속되고 있는 데다 국제 투자은행들이 엔달러 환율 전망을 상향조정하고 있음에도 불구하고 원달러 환율에 대한 조정은 아직 이루어지지 않고 있어 기업 등 시장참가자들의 기대도 원화 절상 쪽으로 편향되어 있는 데 기인한다고 설명하였음.',
 '동 위원은 환율이 중장기적으로 각국의 기초경제여건(fundamental)을 반영하여 움직인다는 점을 감안할 때 현재의 원엔 환율 하락은 일시적인 현상이며 장기적으로 반전될 가능성이 있는 것으로 보인다는 견해를 밝혔으며, 이에 대해 관련부서에서는 동 견해에 동의하면서 우리나라의 경우 시장규모가 작아 일시적인 외부 충격에 의한 영향을 쉽게 받는다는 점을 설명하는 한편, 향후 우리나라와 미국간 내외금리 역전이 나타나고 작년과 같은 대규모 외환 유입은 일어나지 않을 것임을 감안할 때 원엔 환율이 추가 하락할 여지는 점차 축소될 것으로 생각된다고 답변하였음.',
 '다른 일부 위원은 최근 유로지역의 경제성장 실적이 예상보다 저조한 가운데 일각에서는 정책금리 인하론까지 대두되면서 유로화가 크게 절하되고 있다면서 유로화의 달러화에 대한 흐름이 중장기적으로 변화하는 것으로 볼 수 있는지 물었으며, 이에 대해 관련부서에서는 국제 투자은행들이 달러화 대비 유로화 전망을 기존의 강세에서 약세로 바꾸고 있으며 장기적으로 다시 강세로 반전되지는 않을 것이라는 의견이 지배적이라고 설명하였음.',
 '동 위원은 국내 금리가 현 수준에서 유지되고 미국 금리가 상승하는 가운데 달러화 보유에 따른 환차익 기대가 형성될 경우 원화 강세에 대한 시장의 기대가 약세로 반전되면서 자본유출이 확대될 가능성도 배제할 

In [31]:
import pandas as pd
from ekonlpy.sentiment import MPCK

def text2ngram(text):
    mpck = MPCK()
    bef_tokens = mpck.tokenize(text)
    ngrams = mpck.ngramize(bef_tokens)
    return ngrams

In [32]:
for i in range(len(df_t)):
    txt1 = df_t.loc[i, 's2']
    txt2 = df_t.loc[i, 's3']
    
    ngrams1 = ''
    ngrams2 = ''
    
    print(i)
    for t1 in txt1:
        n_t1 = text2ngram(t1)
        ngrams1 = ngrams1 + ','.join(n_t1) + '@'
    for t2 in txt2:
        n_t2 = text2ngram(t2)
        ngrams2 = ngrams2 + ','.join(n_t2) + '@'

    df_t.loc[i, 'ngram_s2'] = ngrams1
    df_t.loc[i, 'ngram_s3'] = ngrams2

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [33]:
df_t.head()

,date,s2,s3,call,ngram_s2,ngram_s3
0,2005-06-09,"[ 외환국제금융 동향과 관련하여, 일부 위원은 우리나라 경제가 일본경제에 비해 더 ...","[ 금융시장 동향과 관련하여, 일부 위원은 M1이 실물경제와 어느 정도의 시차를 두...",0,"엔달러/NNG;환율/NNG;상승/NNG,기대/NNG;원화/NNG;절상/NNG,통화/...","@가계/NNG;대출/NNG;증가/NNG,주택담보대출/NNG;증가/NNG,주택가격/N..."
1,2005-06-23,[],[],0,,
2,2005-07-07,"[ 외환국제금융 동향과 관련하여, 일부 위원은 6월 들어 국제유가가 크게 오르고 원...","[ 금융시장 동향과 관련하여, 일부 위원은 단기적으로 대출이 급격히 증가한 후에는 ...",0,"원화/NNG;절하/NNG;압력/NNG,가격/NNG;약세/NNG,원화/NNG;약세/N...",민간/NNG;소비/NNG;부진/NNG@@
3,2005-08-11,"[ 외환국제금융 동향과 관련하여, 일부 위원은 위안화 절상 이후 아시아 통화중 우리...","[ 금융시장 동향과 관련하여, 일부 위원은 최근 장기시장금리 상승에도 불구하고 기업...",0,"절상/NNG;원화/NNG;절상/NNG@절상/NNG;위안/NNG;절상/NNG,원화/N...","기업/NNG;자금/NNG;조달/NNG;확대/NNG,기업/NNG;자금/NNG;조달/N..."
4,2005-09-08,"[ 외환국제금융 동향과 관련하여, 일부 위원은 최근 인도네시아 금융불안 사태가 발생...","[ 금융시장 동향과 관련하여, 일부 위원은 시장이 제대로 작동한다면 비우량등급의 회...",0,"@경상/NNG;수지/NNG;흑자/NNG,투자/NNG;매우/MAG;부진/NNG@fed...","@부동산/NNG;가격/NNG;상승/NNG@@경제/NNG;견조/NNG;성장/NNG,금..."


In [34]:
df_t.to_excel('금통위 의사록_마스터2.xlsx')